In [6]:
import pandas as pd
from faker import Faker

fake = Faker()

def generate_synthetic_data(num_entries):
    data = {
        "patient_id": [],
        "first_name": [],
        "last_name": [],
        "gender": [],
        "age": [],
        "height": [],
        "weight": [],
        "address": [],
        "phone_number": [],
        "email": [],
        "diagnosis": [],
        "treatment": [],
        "appointment_date": []
    }

    for _ in range(num_entries):
        data["patient_id"].append(fake.uuid4())
        data["first_name"].append(fake.first_name())
        data["last_name"].append(fake.last_name())
        data["gender"].append(fake.random_element(elements=("Male", "Female", "Other")))
        data["age"].append(fake.random_int(min=0, max=100))
        data["height"].append(fake.random_int(min=140, max=200))  # Height in cm
        data["weight"].append(fake.random_int(min=40, max=150))   # Weight in kg
        data["address"].append(fake.address())
        data["phone_number"].append(fake.phone_number())
        data["email"].append(fake.email())
        data["diagnosis"].append(fake.sentence(nb_words=3))
        data["treatment"].append(fake.sentence(nb_words=5))
        data["appointment_date"].append(fake.date_this_year())

    df = pd.DataFrame(data)
    df.to_csv("../Data/synthetic_ehr_data.csv", index=False)
    return df

# Example usage
synthetic_data = generate_synthetic_data(10000)


In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess_data(ehr_data):
    # Handle missing values
    ehr_data = ehr_data.fillna(method='ffill')

    # Standardize numerical features
    scaler = StandardScaler()
    numerical_features = ["age", "height", "weight"]
    ehr_data[numerical_features] = scaler.fit_transform(ehr_data[numerical_features])

    # Encode categorical features
    encoder = OneHotEncoder()
    categorical_features = ["gender", "diagnosis", "treatment"]
    encoded_features = encoder.fit_transform(ehr_data[categorical_features])
    encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out())
    ehr_data = ehr_data.join(encoded_df).drop(columns=categorical_features)
    
    return ehr_data

# Example usage
ehr_data = pd.read_csv("../data/synthetic_ehr_data.csv")
preprocessed_data = preprocess_data(ehr_data)


C:\Users\Jeyanth\AppData\Local\Temp\ipykernel_30212\3355373923.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ehr_data = ehr_data.fillna(method='ffill')


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def train_model(ehr_data, target):
    X = ehr_data.drop(columns=[target])
    y = ehr_data[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    return model

# Example usage
model = train_model(preprocessed_data, target="target_column")


KeyError: "['target_column'] not found in axis"